In [7]:
#Getting the dataset, in this instance the tiny shakespeare dataset
import requests
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text


In [9]:
print(text)


print("Length of the text: ", len(text))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [10]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [11]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [14]:
#create the encoder and decoder 
stoi = { ch: i for i,ch in enumerate(chars)}
print(stoi)
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join(itos[i] for i in l)


{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [49]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:1000])


torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57

In [50]:
#Data is split into training and validation
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[:n]




In [51]:
block_size = 8

train_data[:block_size+1]
#What this says when plugged into transformer is that it will
#make a prediction for each position in the array
#Context: 18, then 47 comes next
#Context: 18,47 then 56 comes next
#And so on...
#It is predicting the next character up to block size
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    #Gets the chunks of characters up to block size
    context = x[:t+1]
    target = y[t]
    #print(f"when input is {context} the target: {target}")



In [26]:
import torch
torch.manual_seed(1337)

batch_size = 5
block_size = 8

def get_batch(split):
    #Generate as small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')

print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print("\n\n")
#Goes through the batches
for b in range(batch_size):
    #Goes through the blocks
    for t in range(block_size):
        #All the numbers up to the target
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")




inputs:
torch.Size([5, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54],
        [57, 43, 60, 43, 52,  1, 63, 43]])
targets:
torch.Size([5, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39],
        [43, 60, 43, 52,  1, 63, 43, 39]])



when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the

In [81]:
#Now we will start feeding our data into a neural network
import torch.nn as nn
import torch
from torch.nn import functional as f 

class BigramLanguageModel (nn.Module):

    def __init__(self,vocab_size):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self,idx, targets=None):
        logits = self.token_embedding_table(idx) #(B,T,C)

        if targets is None:
            loss = None
        else:
            #Idx and targets are both (B,T) tensor of integers
            #Cross entropy loss function
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = f.cross_entropy(logits, targets)

        return logits,loss

    def generate(self,idx, max_new_tokens):

        for _ in range(max_new_tokens):
            #get predictions
            logits, loss = self(idx)
            #focus only on the final time step
            logits = logits[:,-1,:]
            #apply softmax to get probabilities
            probs = f.softmax(logits, dim=-1)
            #apply sampling to distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            #append sampled index to output
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        
        return idx

bigramModel = BigramLanguageModel(vocab_size)
logits,loss = bigramModel(xb, yb)
print(loss)
#Small, 1x1 tensor that is the index which we will feed
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(bigramModel.generate(idx,max_new_tokens=100)[0].tolist()))
#[0].toList()


tensor(4.7905, grad_fn=<NllLossBackward0>)

U?cI&QF!'XVty&rcwtQMTzIiCLkQ.v,IEF';JFGgZdjOVUt,P'X?&s.oJz,XhiDYEFGzG?,'MnfwaJCH;A?Rew$lLNCSH;bfVJ3l


In [44]:
#Now we will train the model so that the output won't be random nonsense
optimizer = torch.optim.AdamW(bigramModel.parameters(), lr=1e-3)
#Take the gradients and update parameters based on the gradients



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
batch_size = 32
iterations = 10000
for steps in range(iterations):
    #sample a batch of data
    xb,yb = get_batch('train')

    #evaluate the loss
    logits, loss = bigramModel(xb,yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(steps, loss.item())




0 4.593716144561768
1 4.584685325622559
2 4.524739742279053
3 4.710486888885498
4 4.617640018463135
5 4.543707370758057
6 4.6247992515563965
7 4.638001918792725
8 4.567185401916504
9 4.547190189361572
10 4.683438301086426
11 4.598638534545898
12 4.511977195739746
13 4.601484298706055
14 4.635591983795166
15 4.489786624908447
16 4.520582675933838
17 4.606016159057617
18 4.517637729644775
19 4.498356819152832
20 4.481322765350342
21 4.5029754638671875
22 4.556520938873291
23 4.459118843078613
24 4.432202339172363
25 4.570502758026123
26 4.534000873565674
27 4.597680568695068
28 4.557682514190674
29 4.397546768188477
30 4.4183220863342285
31 4.467007160186768
32 4.681276798248291
33 4.4600443840026855
34 4.499058723449707
35 4.4766926765441895
36 4.542222499847412
37 4.594516754150391
38 4.543760776519775
39 4.519219875335693
40 4.570126533508301
41 4.534435272216797
42 4.461192607879639
43 4.550563335418701
44 4.562365531921387
45 4.554652690887451
46 4.544169902801514
47 4.4810800552368

Self-attention trick

In [86]:
#consider the following example for learning

torch.manual_seed(1337)
B,T,C = 4,8,2

x = torch.randn(B,T,C)
x.shape
#Context can only flow from earlier in the sequence to later 
#We want the 8 tokens in batch to provide context to each other
#How to provide context: for each tth token in the batch we want to calculate the vectors at all previous tokens and the current one

torch.Size([4, 8, 2])

In [87]:
xbow = torch.zeros((B,T,C))

for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] #everything up to & including t token
        xbow[b,t] = torch.mean(xprev,0)






In [92]:
#Version 2
#Efficient version using matrix multiplication
wei = torch.tril(torch.ones(T,T))

wei = wei / torch.sum(wei, 1, keepdim=True)

xbow2 = wei @ x # (B, T, T) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow2)



True

In [93]:
xbow[0], xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [96]:
#Version 3 
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))

wei = wei.masked_fill(tril == 0, float('-inf'))
wei = f.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [89]:
#Gets lower triangular matrix


tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [91]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
#Normalize the matrix values such that the sum of all values in a row add to 1
a = a / torch.sum(a,1,keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b

print('a=')
print(a)
print('b=')
print(b)
print('c=')
print(c)        


a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
